In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

using Markdown
import Dates: Day

# Import some libraries from the JDP project
using JDP.BugRefs
using JDP.Tracker
using JDP.Spammer
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla
using JDP.Trackers.Redmine
using JDP.Repository
using JDP.Functional

trackers = load_trackers();

args = try
    WEAVE_ARGS
catch
    Dict()
end

In [ ]:
product = get(args, "product", "opensuse-Tumbleweed")
dryrun = get(args, "dryrun", false)
host = get(args, "host", "ooo")
public = get(args, "public", false)

# Bug Tag Propagation

This partially automates copying existing bug tags from one failed test result to another.

## Find untagged test failures

First we fetch the test results from OpenQA.

In [ ]:
# Get some cached job results from the instance specified by `host`
# The final arg limits the number of results which will be loaded
allres = Repository.fetch(OpenQA.TestResult, Vector, host, OpenQA.RecentOrInterestingJobsDef)
md"We now have $(length(allres)) test results"

Next we find the latest build for product which matches our product filter.

In [ ]:
prodbuilds = OpenQA.get_product_builds(filter(r -> occursin(Regex(product), r.product), allres))

[p => [b.val for b in Iterators.take(bs, 5)] for (p, bs) in prodbuilds]

Then filter the results to only include fails from a particular build with no bug references.

In [ ]:
untagged = filter(allres) do res
    (builds = get(prodbuilds, res.product, nothing)) ≠ nothing &&
    first(builds).orig == res.build &&
    occursin(r"failed", res.result) &&
    (isempty(res.refs) || all(rf -> rf.negated, res.refs))
end

length(untagged)

## Find existing bug references for tests

Now we search all the test results for bug tags that were added in other builds or architectures.

In [ ]:
fqn = OpenQA.get_fqn

tagdict = Dict(fqn(res) => Dict{BugRefs.Ref, OpenQA.TestResult}() for res in untagged)

for res in allres
    if haskey(tagdict, fqn(res)) && length(res.refs) > 0
        refs = tagdict[fqn(res)]
        
        for rf in res.refs
            # Negated bugrefs permanently stop a bugref from being used on the same test
            if rf.negated
                delete!(refs, BugRefs.Ref(rf.tracker, rf.id, false))
                refs[rf] = res
            elseif !haskey(refs, BugRefs.Ref(rf.tracker, rf.id, true))
                refs[rf] = res
            end
        end
    end
end

# Anti-tags are not propogated for the same reason graveyards don't scale
for refs in values(tagdict)
    filter!(p -> !p.first.negated, refs)
end

length(tagdict)

Next we fetch the bug data for each bug reference. This allows further filtering.

In [ ]:
# Always match for tracker items which don't specify the arch
arch_matches(b, arch::String) = true

# If a bug doesn't have an arch set or we don't recognise it, then we assume the arch matches
function arch_matches(b::Redmine.Bug, arch::String)
    arch_tags = filter(in(["aarch64", "ppc64le", "x86_64", "s390x"]), Redmine.tags(b))
    isempty(arch_tags) || arch in arch_tags
end

arch_map = Dict("x86-64" => "x86_64", 
                "aarch64" => "aarch64",
                "PowerPC-64" => "ppc64le",
                "S/390-64" => "s390x")

function arch_matches(b::Bugzilla.Bug, arch::String)
    !haskey(arch_map, arch) || arch_map[arch] == arch
end

In [ ]:
bugdict = Dict(name => Tuple{BugRefs.Ref, Any, OpenQA.TestResult, OpenQA.TestResult}[] for name in keys(tagdict))

for res in untagged
    for (rf, orig_res) in tagdict[fqn(res)]
        bug = Repository.fetch(rf)
        
        arch_matches(bug, res.arch) || continue
        
        push!(bugdict[fqn(res)], (rf, bug, res, orig_res))
    end
end

length(bugdict)

In [ ]:
mdbuf = IOBuffer()

for (k, v) in pairs(bugdict)
    if isempty(v)
        continue
    end
    
    println(mdbuf, "- ", k)
    for (rf, bug, res, orig_res) in v
        print(mdbuf, "    * ")
        show(mdbuf, MIME("text/markdown"), rf)
        print(mdbuf, " ")
        if bug != nothing
            show(mdbuf, MIME("text/markdown"), bug)
        else
            print(mdbuf, "*no data*")
        end
        println(mdbuf)
    end
    
end

seek(mdbuf, 0)
Markdown.parse(mdbuf)

## Posting back to OpenQA

Now we can post the results back to OpenQA. We create two sets of tags; one set we will post back as full tags, the other we will post as advisory notices. The advisory notices will not be recognised by OpenQA, or anything else, as legitimate tags.

In [ ]:
taggings = Dict()
advisory = Dict()

advisory_statuses = ["RESOLVED", "Resolved", "Feedback", "Closed", "Rejected"]

for  v = values(bugdict), (rf, bug, res, orig_res) = v
    tags = if bug.status in advisory_statuses
        get!(advisory, res.job.id, [])
    else
        get!(taggings, res.job.id, [])
    end
    
    push!(tags, (res, rf, bug, orig_res))
end
        
"$(length(taggings)) full tags and $(length(advisory)) advisory notices"     

In [ ]:
oqa = get_tracker(trackers, host)
ses = Tracker.ensure_login!(oqa)
modified_jobs = Set{Int64}()
pages_host = public ? "https://palethorpe.gitlab.io" : "https://rpalethorpe.io.suse.de"

if dryrun
    @warn "Nothing will be posted because dryrun is set"
end

for (jid, refs) in taggings
    mdbuf = IOBuffer()
    
    print(mdbuf,
        "This is an automated message from the [JDP Propagate Bug Tags]($pages_host/jdp/reports/Propagate%20Bug%20Tags.html) report",
        "\n\n",
        "The following bug tags have been propagated: \n\n")
    for (res, rf, bug, orig_res) in refs
        print(mdbuf, "- `", res.name, "`", rf.negated ? ":" : ": ", rf)
        if rf.negated
            print(mdbuf, " *This is an anti-tag to prevent the following bug being used again*")
        end
        print(mdbuf, " [")
        show(mdbuf, MIME("text/markdown"), bug)
        println(mdbuf, "]")
        print(mdbuf, "    + From ")
        show(mdbuf, MIME("text/markdown"), orig_res)
        println(mdbuf)
    end
    
    text = String(take!(mdbuf))
    
    @info "Posting comment to job $jid" text
    display(Markdown.parse(text))
    if !dryrun
        OpenQA.post_job_comment(ses, jid, text)
        push!(modified_jobs, jid)
    end
end

for (jid, refs) in advisory
    mdbuf = IOBuffer()
    
    print(mdbuf,
        "This is an automated message from the [JDP Propagate Bug Tags]($pages_host/jdp/reports/Propagate%20Bug%20Tags.html) report",
        "\n\n",
        "The following tags have not been propagated, but may be of interest: \n\n")
    for (res, rf, bug, orig_res) in refs
        print(mdbuf, "- `", res.name, "`: ", rf.tracker.tla, "@", rf.id, " [")
        show(mdbuf, MIME("text/markdown"), Repository.fetch(rf))
        println(mdbuf, "]")
        print(mdbuf, "    + From ")
        show(mdbuf, MIME("text/markdown"), orig_res)
        println(mdbuf)
    end
    
    text = String(take!(mdbuf))
    
    @info "Posting comment to job $jid" text
    display(Markdown.parse(text))
    if !dryrun
        try
            OpenQA.post_job_comment(ses, jid, text)
            push!(modified_jobs, jid)
        catch error
            @error "Error while trying to post comment" error
        end
    end
end

## Notifications

Send out notifications if some tags were propagated.

In [ ]:
if !isempty(taggings) || !isempty(untagged)
    spam = IOBuffer()
    
    if !isempty(taggings)
        println(spam, length(taggings), " bug tags propagated on:")
        for (p, bs) in prodbuilds
            println(spam, "  * ", p, " build ", first(bs).orig)
        end
    end
    
    ulen = sum(untagged) do res
        haskey(taggings, res.name) ? 0 : 1
    end
    if ulen > 0
        flag_name = "notify-untagged"
        flag_val = join((first(bs).orig for bs in values(prodbuilds)), "-")
        if flag_val ≠ Repository.get_temp_flag(flag_name)
            println(spam, ulen, " failing tests are still missing bug tags!")
            Repository.set_temp_flag(flag_name, flag_val, Day(2))
        end
    end
    
    if position(spam) > 0
        println(spam, "See the [Propagate Bug Tags]($pages_host/jdp/reports/Propagate%20Bug%20Tags.html) report for details.")
    end    
    text = String(take!(spam))
    
    if dryrun
        display(Markdown.parse(text))
    elseif !isempty(text)
        Spammer.post_message(Spammer.Message(text, ["rpalethorpe"]))
    end
end

## Refresh effected jobs

Redownload the comments for jobs which we just tried to post to.

In [ ]:
let n = length(modified_jobs)
    @info "$host: Refreshing $n job comments"

    for (i, jid) in enumerate(modified_jobs)
        @info "$host: GET job $jid ($i of $n)"
        
        comments = try
            OpenQA.json_to_comments(OpenQA.get_job_comments(ses, jid))
        catch
            nothing
        end
        
        if comments != nothing
            job = Repository.fetch(OpenQA.JobResult, host, jid)
            job.comments = comments
            Repository.store("$host-job-$(jid)", job)
        end
    end
end